In [10]:
import os

if 'original_dir' not in globals() :
    original_dir = os.getcwd()
    original_dir = os.path.dirname(os.path.dirname((os.path.dirname(original_dir))))
    os.chdir(original_dir)
from setup.default import *
import scipy.stats as stats


In [11]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')
menu_w_sold_out['menu_no'] = menu_w_sold_out.groupby(['day', 'meal_time'])['day'].transform('size')
df = menu_w_sold_out[(menu_w_sold_out.meal_time =='dinner') & 
                     (menu_w_sold_out.menu_no>1)&
                     (menu_w_sold_out.day>='2023-06-01')]


In [12]:
df = menu_w_sold_out[(menu_w_sold_out.meal_time =='dinner') & 
                     (menu_w_sold_out.menu_no>1)&
                     (menu_w_sold_out.day>='2023-06-01')]
df.shape

(372, 13)

In [13]:
# 결측치 처리
df.loc[:,'course_kcal'] = df['course_kcal'].fillna(df['course_kcal'].mean().astype(int))
df.loc[:,'course_protein'] = df['course_protein'].fillna(df['course_protein'].mean().astype(int))
df.loc[:, 'course_na'] = df['course_na'].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df.loc[:,'course_na'] = df['course_na'].fillna(df['course_na'].mean().astype(int))

# 특징과 타겟 변수 설정
df['day'] = pd.to_datetime(df['day'])

test_data = df[(df['day'] >= '2024-04-01')]
train_data = df[(df['day'] < '2024-04-01')]

# 피처와 타겟 분리
X_train = train_data.drop(columns=['post_no','day','meal_time','is_soldout', 'course', 'soldout','soldout_time'])
y_train = train_data['is_soldout'].astype(int)

X_test = test_data.drop(columns=['post_no','day','meal_time','is_soldout', 'course', 'soldout','soldout_time'])
y_test = test_data['is_soldout'].astype(int)

# 범주형 변수 인코딩 및 스케일링
numeric_features = ['course_kcal', 'course_protein', 'course_na']
categorical_features = ['day_of_week', 'course_no']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

/var/folders/m7/m0z7cct103x4s_jnq77y81mw0000gn/T/ipykernel_19202/49068104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day'] = pd.to_datetime(df['day'])


In [14]:
# 로지스틱 회귀 모델 파이프라인
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [15]:
# 모델 훈련
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['course_kcal',
                                                   'course_protein',
                                                   'course_na']),
                                                 ('cat', OneHotEncoder(),
                                                  ['day_of_week',
                                                   'course_no'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [16]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [17]:

# 예측
y_pred = model.predict(X_test)

# 결과 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.65625
              precision    recall  f1-score   support

           0       0.66      1.00      0.79        63
           1       0.00      0.00      0.00        33

    accuracy                           0.66        96
   macro avg       0.33      0.50      0.40        96
weighted avg       0.43      0.66      0.52        96



/Users/al01989093/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/al01989093/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/al01989093/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
